In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article
import time
import random

In [2]:
def make_urllist_sub(index, date, page):
    sub_urllist = []
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(index)+'&date='+str(date)+'&page='+str(page)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
    news = requests.get(url, headers=headers)
    soup = BeautifulSoup(news.content, 'html.parser')
    news_list = soup.select('.newsflash_body .type06_headline li dl')
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))
    for line in news_list:
        sub_urllist.append(line.a.get('href'))
    return sub_urllist

In [3]:
def make_urllist(index, date):
    total_url_list = []
    temp_url = []
    for page in range(1, 1000):
        url_list = make_urllist_sub(index, date, page)
        if url_list[-1] == temp_url:
            break
        temp_url = url_list[-1]
        total_url_list.extend(url_list)
    return total_url_list

In [4]:
def make_data(urllist, index, date):
    title_list = []
    text_list = []
    item_list = []
    for url in urllist:
        temp_sen = url.split('=')
        item_list.append(temp_sen[-1])
        article = Article(url, language='ko')
        article.download()
        article.parse()
        title_list.append(article.title)
        text_list.append(article.text)
    df = pd.DataFrame({'title': title_list, 'permalink': urllist, 'item_id': item_list, 'contents': text_list, 'section_id': index, 'datetime':date})
    return df

In [5]:
def make_total_data(index, startday):
    t = time.time()
    df = None
    temp_url = []
    for date in range(startday, startday+7): # 기본 설정은 1주일치로 되어있습니다.
        url_list = make_urllist(index, date)
        if temp_url == url_list[-1]:
            break
        temp_url = url_list[-1]
        df_temp = make_data(url_list, index, date)
        if df is not None:
            df = pd.concat([df, df_temp])
        else:
            df = df_temp
        df.to_csv(str(date) + '.csv') # 1일치가 끝날때마다 '해당 날짜.csv'로 저장됩니다.
        time.sleep(random.uniform(5, 10)) # 1일치가 끝나면 5초에서 10초동안 쉬도록 설정되어있습니다.
    df.to_csv(str(date) + '-' + str(date+7) + '.csv') #1주일치가 끝나면 '첫날-마지막날.csv'로 저장됩니다.
    print(time.time()-t)
    return df

In [6]:
index = 100 #인덱스 설정은 개인별로 바꾸시길 바랍니다.

## index

### 100 : 정치
### 101 : 경제
### 102 : 사회
### 103 : 생활/문화
### 104 : 세계
### 105 : IT/과학
### 110 : 오피니

In [7]:
startday = 20200507 #2020년 05월 06일

In [8]:
df = make_total_data(index, startday)
df

715.3750755786896


,title,permalink,item_id,contents,section_id,datetime
0,北은 ICBM 쐈는데... 우리 軍은 단거리 미사일도 실패,https://news.naver.com/main/read.nhn?mode=LSD&...,0003529340,"지난 2017년 사거리 800㎞, 탄두 중량 500㎏의 현무-2C 탄도미사일이 발사...",100,20200507
1,대선주자 인기 반영?…이낙연 의원실 5급 비서관 경쟁률 111대1,https://news.naver.com/main/read.nhn?mode=LSD&...,0011594700,민주당 이낙연 위원장 (서울=연합뉴스) 김인철 기자 = 더불어민주당 이낙연 코로나1...,100,20200507
2,"美 의회조사국 “北, 핵무기 포기해야 美 핵 공격 대상서 제외될 것”",https://news.naver.com/main/read.nhn?mode=LSD&...,0003284861,경기도 파주시 오두산 통일전망대에서 바라본 북한 황해북도 개풍군 마을이 고요한 모습...,100,20200507
3,駐멕시코대사 서정인·駐호주대사 강정식,https://news.naver.com/main/read.nhn?mode=LSD&...,0004570727,서정인 전 한·아세안 특별정상회의 준비기획단장이 주멕시코 대사로 부임한다. 강정식 ...,100,20200507
4,"'재수생' 김태년, 당선 소감때 북받쳐… 민주 원내대표 경선 안팎",https://news.naver.com/main/read.nhn?mode=LSD&...,0003463150,이해찬 더불어민주당 대표가 7일 오후 서울 여의도 국회 의원회관에서 열린 원내대표 ...,100,20200507
...,...,...,...,...,...,...
3140,“아스팔트 보수와 결별할 것”,https://news.naver.com/main/read.nhn?mode=LSD&...,0002998607,통합당 원내대표 3인. 그래픽=김은교 kim.eungyo@joongang.co.kr...,100,20200507
3141,“여당 독주 땐 여론 힘으로 제동”,https://news.naver.com/main/read.nhn?mode=LSD&...,0002998606,통합당 원내대표 3인. 그래픽=김은교 kim.eungyo@joongang.co.kr...,100,20200507
3142,[TF이슈] 통합당 막판 대여 공세 집중…'오거돈 성추행' 검찰고발,https://news.naver.com/main/read.nhn?mode=LSD&...,0000024936,미래통합당이 직원 성추행으로 사퇴한 오거돈 전 시장과 관련해 검찰 고발에 나서면서 ...,100,20200507
3143,[TF초점] 돌아온 北 김정은 비핵화 대화로 복귀할까?,https://news.naver.com/main/read.nhn?mode=LSD&...,0000024935,'건강 이상설'을 뚫고 등장한 김정은 북한 국무위원장이 우리 정부가 재추진하고 있는...,100,20200507
